# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat ws-tech-reports.tsv

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("ws-tech-reports.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2019-08-26,Touched by the Hackathon: A Study on the Conne...,The 2nd ACM SIGSOFT International Workshop on ...,"Time-bounded events such as hackathons, code f...",Alexander Nolte. 2019. Touched by the Hackatho...,SIGSOFT-workshop-nolte-2019,https://eipapa.github.io/hackathon-planning-ki...
1,2017-02-25,Hacking and Making at Time-Bounded Events: Cur...,CSCW'17,Time-bounded collaborative events in which tea...,"Filippova, A., Chapman, B., Geiger, R. S., Her...",CSCW-workshop-filippova-2017,http://eipapa.github.io/hackathon-planning-kit...
2,2018-01-04,The 2nd Workshop on Hacking and Making at Time...,Extended Abstracts of the 2018 ACM CHI Confere...,"Hackathons or Hackathon-style events, describe...","Pe-Than, E.P.P., Herbsleb, J.D., Nolte, A., Ge...",CHI-workshop-2018,http://eipapa.github.io/hackathon-planning-kit...
3,2018-01-09,An Analysis of the PEARC 2018 Science Gateways...,Practice & Experience in Advanced Research Com...,Science Gateways are web portals on which data...,"Powell, J., Hayden, L., Nolte, A., Herbsleb, J...",SGCI-poster-2018,http://eipapa.github.io/hackathon-planning-kit...
4,2018-12-30,Technical Report for the 2nd Workshop on Hacki...,Technical Report CMU-ISR-18-109,"In hackathons, small teams work together over ...","Pe-Than, E.P.P., and Nolte, A. (Editors) (2018...",ISR-techreport-2018,http://eipapa.github.io/hackathon-planning-kit...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += """collection: wsandtechreports"""
    md += """\npermalink: /wsandtechreports/""" + html_filename
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    md += "\ndate: " + str(item.pub_date) 
    md += "\nvenue: '" + html_escape(item.venue) + "'"    
#     if len(str(item.paper_url)) > 5:
#         md += "\npaperurl: '" + item.paper_url + "'"
#     md += "\ncitation: '" + html_escape(item.citation) + "'"
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.paper_url)) > 5:
#         md += "\n[<span style=\"color: #c41e3a\">Download PDF here.</span>](" + item.paper_url + ")\n"
        md += "\n[Download PDF here.](" + item.paper_url + ")\n"
    if len(str(item.excerpt)) > 5:
        md += "\n**Abstract**:" + html_escape(item.excerpt) + "\n"    
#     md += "\nAbstract: " + html_escape(item.description) + "\n"
    md += "\n**Recommended citation**: " + item.citation
    md_filename = os.path.basename(md_filename)
    with open("../_wsandtechreports/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_wsandtechreports/

2017-02-25-CSCW-workshop-filippova-2017.md
2018-01-04-CHI-workshop-2018.md
2018-01-09-SGCI-poster-2018.md
2018-12-30-ISR-techreport-2018.md
2019-08-26-SIGSOFT-workshop-nolte-2019.md


In [ ]:
!cat ../_publications/2009-10-01-paper-title-number-1.md